# News data collection

In [1]:
import sys
sys.path.append(r'C:\Users\jessi\OneDrive\1 Projects\1-svts') # Relocate Jupyter Notebook working directory

from src.utils import news_data

# All capital letters for the parameters
df = news_data.extract_sentiment(ticker='FOREX:EUR', date_from='20230404T0130', sort='LATEST')

print(df)

        ticker relevance_score ticker_sentiment_score ticker_sentiment_label  \
0    FOREX:EUR        0.554916               0.254518       Somewhat-Bullish   
1    FOREX:EUR        0.096671              -0.026837                Neutral   
2    FOREX:EUR        0.173958               0.062668                Neutral   
3    FOREX:EUR        0.147818               0.060182                Neutral   
4    FOREX:EUR        0.236972               0.252242       Somewhat-Bullish   
..         ...             ...                    ...                    ...   
761  FOREX:EUR        0.194763               0.148621                Neutral   
762  FOREX:EUR        0.062597              -0.053192                Neutral   
763  FOREX:EUR        0.075161               0.129358                Neutral   
764  FOREX:EUR        0.152178               0.265874       Somewhat-Bullish   
765  FOREX:EUR        0.146618               0.248585       Somewhat-Bullish   

          time_published(est)  
0   202

In [7]:
# Save csv file to folder
eur_sentiment = df.to_csv(r'C:\Users\jessi\OneDrive\1 Projects\1-svts\data\external\news-analysis\eur_sentiment.csv', index=False)

## Market execution


In [18]:
from twisted.internet import reactor
import json
from ctrader_fix import *
import datetime

In [39]:
# you can use two separate config files for QUOTE and TRADE
with open("config-quote.json") as configFile:
    config = json.load(configFile)

In [40]:
client = Client(config["Host"], config["Port"], ssl = config["SSL"])

In [35]:
def send(request):
    diferred = client.send(request)
    diferred.addCallback(lambda _: print("\nSent: ", request.getMessage(client.getMessageSequenceNumber()).replace("", "|")))

In [17]:
def onMessageReceived(client, responseMessage): # Callback for receiving all messages
    print("\nReceived: ", responseMessage.getMessage().replace("", "|"))
    # We get the message type field value
    messageType = responseMessage.getFieldValue(35)
    # we send a security list request after we received logon message response
    if messageType == "A":
        securityListRequest = SecurityListRequest(config)
        securityListRequest.SecurityReqID = "A"
        securityListRequest.SecurityListRequestType = 0
        send(securityListRequest)
    # After receiving the security list we send a market order request by using the security list first symbol
    elif messageType == "y":
        # We use getFieldValue to get all symbol IDs, it will return a list in this case
        # because the symbol ID field is repetitive
        symboldIds = responseMessage.getFieldValue(55)
        if config["TargetSubID"] == "TRADE":
            newOrderSingle = NewOrderSingle(config)
            newOrderSingle.ClOrdID = "B"
            newOrderSingle.Symbol = symboldIds[1]
            newOrderSingle.Side = 1
            newOrderSingle.OrderQty = 1000
            newOrderSingle.OrdType = 1
            newOrderSingle.Designation = "From Jupyter"
            send(newOrderSingle)
        else:
            marketDataRequest = MarketDataRequest(config)
            marketDataRequest.MDReqID = "a"
            marketDataRequest.SubscriptionRequestType = 1
            marketDataRequest.MarketDepth = 1
            marketDataRequest.NoMDEntryTypes = 1
            marketDataRequest.MDEntryType = 0
            marketDataRequest.NoRelatedSym = 1
            marketDataRequest.Symbol = symboldIds[1]
            send(marketDataRequest)
    # after receiving the new order request response we stop the reactor
    # And we will be disconnected from API
    elif messageType == "8" or messageType == "j":
        print("We are done, stopping the reactor")
        reactor.stop()

def disconnected(client, reason): # Callback for client disconnection
    print("\nDisconnected, reason: ", reason)

def connected(client): # Callback for client connection
    print("Connected")
    logonRequest = LogonRequest(config)
    send(logonRequest)
    
# Setting client callbacks
client.setConnectedCallback(connected)
client.setDisconnectedCallback(disconnected)
client.setMessageReceivedCallback(onMessageReceived)

In [21]:
# Creating a Python module to fetch market data using FIX protocol.
# This is a simplified example and assumes that you have a client object and config dictionary initialized.

class MarketDataFetcher:
    def __init__(self, client, config):
        self.client = client
        self.config = config
        self.client.setConnectedCallback(self.connected)
        self.client.setDisconnectedCallback(self.disconnected)
        self.client.setMessageReceivedCallback(self.onMessageReceived)

    def send(self, request):
        deferred = self.client.send(request)
        deferred.addCallback(lambda _: print("\nSent: ", request.getMessage(self.client.getMessageSequenceNumber()).replace("", "|")))

    def onMessageReceived(self, client, responseMessage):
        print("\nReceived: ", responseMessage.getMessage().replace("", "|"))
        messageType = responseMessage.getFieldValue(35)
        if messageType == "A":
            self.send_security_list_request()
        elif messageType == "y":
            self.send_market_data_request(responseMessage)

    def disconnected(self, client, reason):
        print("\nDisconnected, reason: ", reason)

    def connected(self, client):
        print("Connected")
        self.send_logon_request()

    def send_logon_request(self):
        logonRequest = LogonRequest(self.config)
        self.send(logonRequest)

    def send_security_list_request(self):
        securityListRequest = SecurityListRequest(self.config)
        securityListRequest.SecurityReqID = "A"
        securityListRequest.SecurityListRequestType = 0
        self.send(securityListRequest)

    def send_market_data_request(self, responseMessage):
        symbolIds = responseMessage.getFieldValue(55)
        marketDataRequest = MarketDataRequest(self.config)
        marketDataRequest.MDReqID = "a"
        marketDataRequest.SubscriptionRequestType = 1
        marketDataRequest.MarketDepth = 1
        marketDataRequest.NoMDEntryTypes = 1
        marketDataRequest.MDEntryType = 0
        marketDataRequest.NoRelatedSym = 1
        marketDataRequest.Symbol = symbolIds[1]
        self.send(marketDataRequest)

# Example usage:
# market_data_fetcher = MarketDataFetcher(client, config)
# Note: The client and config objects need to be initialized before passing them to MarketDataFetcher.
market_data_fetcher = MarketDataFetcher(client, config)

In [43]:
market_data_fetcher.send_security_list_request()

# Market data manual input

In [ ]:
# Combine all the steps into a single cell to transform the TSV file and save it as a CSV file

# Import required libraries
import pandas as pd
from pytz import timezone

# Read the TSV file
tsv_file_path = '/mnt/data/EURUSD_M1_202310180000_202310192047.csv'
df_tsv = pd.read_csv(tsv_file_path, sep='\t')

# Drop the <VOL> column
df_tsv.drop('<VOL>', axis=1, inplace=True)

# Combine <DATE> and <TIME> columns into a single DateTime column and convert to datetime object
df_tsv['DateTime'] = pd.to_datetime(df_tsv['<DATE>'] + ' ' + df_tsv['<TIME>'])

# Drop the separate <DATE> and <TIME> columns
df_tsv.drop(['<DATE>', '<TIME>'], axis=1, inplace=True)

# Convert time from UTC+3 to EST (UTC-4)
utc3 = timezone('Etc/GMT-3')
utc4 = timezone('Etc/GMT+4')
df_tsv['DateTime'] = df_tsv['DateTime'].apply(lambda x: utc3.localize(x).astimezone(utc4))

# Reorder columns
df_tsv = df_tsv[['DateTime', '<OPEN>', '<HIGH>', '<LOW>', '<CLOSE>', '<TICKVOL>', '<SPREAD>']]

# Save as CSV
output_csv_file_path = '/mnt/data/EURUSD_M1_20231018_20231019_modified_single_cell.csv'
df_tsv.to_csv(output_csv_file_path, index=False)

output_csv_file_path
